In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

#Collect Reviews

In [14]:
#Collect Reviews

response = requests.get('https://www.airlinequality.com/airline-reviews/vueling-airlines/')
soup = BeautifulSoup(response.text, 'html.parser')
review_elements = soup.find_all('div', class_='text_content')
reviews = []
for element in review_elements:
    review = element.get_text().strip()
    reviews.append(review)

In [15]:
# Print the extracted reviews
for review in reviews:
    print(review)

✅ Trip Verified | Unpleasant Experience with Vueling Airlines: Excessive and Unreasonable Baggage Charges and rude staff. Recent trip from Paris to La Coruña, Spain, the experience was far from the quality service one would expect from a prominent airline operating from one of the world's top tourist destinations. Upon arrival at the airport in Paris, I encountered an agent at the counter who seemed to lack proficiency in English or Spanish. Given Paris's global appeal, I was taken aback by the language barrier, which made communication and problem-solving unnecessarily difficult. My family and I (a group of 6) had already paid for three extra suitcases. Additionally, we had three carry-on bags that conformed to your airline's size specifications. However, the counter agent insisted we had to pay extra for each carry-on to take them onboard. The lack of effective communication compounded the frustration of this unexpected charge. Even when faced with the inconvenience of either losing 

#Data cleaning

In [16]:
updated_reviews = []
for review in reviews:
    # Split the string at the "|" symbol and retrieve the portion after it
    updated_text = review.split("|")[-1].strip()
    updated_reviews.append(updated_text)
    df = pd.DataFrame(np.array(updated_reviews), columns=['review'])

print(df)

                                              review
0  Unpleasant Experience with Vueling Airlines: E...
1  The Vueling website has proven to be deceptive...
2  When we arrived at Gatwick late Saturday eveni...
3  Flight was delayed. Airplane was dirty. Staff ...
4  Shocking service. Avoid this airline. Two week...
5  They knew about the flight delay for 1.5hrs be...
6  Horrible customer service. We bought a ticket ...
7  Absolutely terrible, arrive to check in, 13 pa...
8  Paid for a priority ticket travelling with my ...
9  I will never book with Vueling again. We booke...


#Instantiate Model

In [17]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


#Encode and Calculate Sentiment

In [21]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
print(df)

                                              review  sentiment
0  Unpleasant Experience with Vueling Airlines: E...          2
1  The Vueling website has proven to be deceptive...          1
2  When we arrived at Gatwick late Saturday eveni...          2
3  Flight was delayed. Airplane was dirty. Staff ...          1
4  Shocking service. Avoid this airline. Two week...          1
5  They knew about the flight delay for 1.5hrs be...          1
6  Horrible customer service. We bought a ticket ...          1
7  Absolutely terrible, arrive to check in, 13 pa...          1
8  Paid for a priority ticket travelling with my ...          4
9  I will never book with Vueling again. We booke...          1
